# 4.3 Gap-filling glucose metabolism

Examination of the draft model of EcN's growth using EX_glu_e as a carbon source. Within this notebook the model is checked for
- Growth on glucose
- Growth on other carbon sources

In [1]:
import cobra
import pandas as pd
from cobra.io import load_json_model
from glob import glob
from cobra.manipulation.modify import rename_genes

In [2]:
#Establish a definition that initializes models to an in silico representation of M9 media

def m9(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0 
            
    model.reactions.EX_ca2_e.lower_bound=-1000
    model.reactions.EX_cl_e.lower_bound=-1000
    model.reactions.EX_co2_e.lower_bound=-1000
    model.reactions.EX_cobalt2_e.lower_bound=-1000
    model.reactions.EX_cu2_e.lower_bound=-1000
    model.reactions.EX_fe2_e.lower_bound=-1000
    model.reactions.EX_fe3_e.lower_bound=-1000
    model.reactions.EX_h_e.lower_bound=-1000
    model.reactions.EX_h2o_e.lower_bound=-1000
    model.reactions.EX_k_e.lower_bound=-1000
    model.reactions.EX_mg2_e.lower_bound=-1000
    model.reactions.EX_mn2_e.lower_bound=-1000
    model.reactions.EX_mobd_e.lower_bound=-1000
    model.reactions.EX_na1_e.lower_bound=-1000
    model.reactions.EX_tungs_e.lower_bound=-1000
    model.reactions.EX_zn2_e.lower_bound=-1000
    model.reactions.EX_ni2_e.lower_bound=-1000
    model.reactions.EX_sel_e.lower_bound=-1000
    model.reactions.EX_slnt_e.lower_bound=-1000
    model.reactions.EX_glc__D_e.lower_bound=-20
    model.reactions.EX_so4_e.lower_bound=-1000
    model.reactions.EX_nh4_e.lower_bound=-1000
    model.reactions.EX_pi_e.lower_bound=-1000
    model.reactions.EX_cbl1_e.lower_bound=-.01
    model.reactions.EX_o2_e.lower_bound=-20
       
    return model

# 1. Growth on glucose

In [3]:
# Load EcN model
EcN_ID = 'CP022686.1'
model=cobra.io.load_json_model('../data/models/%s_cur_4.2.json'%EcN_ID)

# Test whether the EcN model can grown on glucose (EX_glc__D_e)
with model:
    m9(model)
    solution = model.optimize()
solution

,fluxes,reduced_costs
ALATA_D2,0.00000,6.938894e-18
SHCHD2,0.00025,0.000000e+00
CPPPGO,0.00025,-1.734723e-17
GTHOr,0.00000,-8.673617e-18
DHORD5,0.00000,0.000000e+00
...,...,...
MALt5,0.00000,0.000000e+00
SALCHS2FEexs,0.00000,0.000000e+00
SALCHS4FEexs,0.00000,0.000000e+00
DHPTDCs,0.00000,0.000000e+00


In [4]:
# The base model can synthesize biomass in this condition
base=cobra.io.load_json_model('../data/models/iML1515.json')
with base:
    m9(base)
    solution = base.optimize()
solution

,fluxes,reduced_costs
ALATA_D2,0.000000,-2.104544e-02
SHCHD2,0.000250,-5.551115e-17
CPPPGO,0.000250,-5.551115e-17
GTHOr,0.277377,0.000000e+00
DHORD5,0.000000,-2.775558e-17
...,...,...
SUCCt1pp,0.000000,0.000000e+00
QUINDH,0.000000,-8.418177e-03
LCARSyi,0.000000,-8.418177e-03
BIOMASS_Ec_iML1515_core_75p37M,1.120796,-6.335180e-16


### Use gapfill to find missing reactions for glucose consumption

In [5]:
universal = cobra.Model("universal_reactions")
for rxn_obj in base.reactions:
    if rxn_obj.id not in model.reactions:
        universal.add_reaction(rxn_obj.copy())


gapfiller = cobra.flux_analysis.gapfilling.GapFiller(model, universal=universal, demand_reactions=False, integer_threshold=1e-12)
gapfiller.model.solver.configuration.tolerances.feasibility = 1e-16
gapfiller.model.solver.configuration.tolerances.integrality = 1e-16
gapfiller.model.solver.configuration.tolerances.optimality = 1e-16
result = gapfiller.fill()
result

[[]]

In [6]:
# Check the growth rate with missing reactions added (if reactions are missing)
modelCopy = model.copy()
for rxn in result[0]:
    print('Reaction to add:', rxn)
    modelCopy.add_reaction(rxn)

m9(modelCopy)
modelCopy.reactions.EX_glc__D_e.lower_bound=-20
modelCopy.optimize()

,fluxes,reduced_costs
ALATA_D2,0.00000,-2.105453e-02
SHCHD2,0.00025,0.000000e+00
CPPPGO,0.00025,0.000000e+00
GTHOr,0.00000,0.000000e+00
DHORD5,0.00000,-1.110223e-16
...,...,...
MALt5,0.00000,0.000000e+00
SALCHS2FEexs,0.00000,0.000000e+00
SALCHS4FEexs,0.00000,0.000000e+00
DHPTDCs,0.00000,0.000000e+00


No reactions are missing for growth on glucose

# 2. Other carbon sources

In [7]:
# Find out missing reactions for growth on:
# Glucose, Arabinose, Fucose, Galactose, Gluconate, Mannose, N-acetylgalactosamine, N-acetylneuraminate > https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4510460/#!po=30.0000
# Galacturonate, Xylose, Acetate, Fructose, Malate, Maltose, Ribose, Glucuronic, Lactose, N-acetylglucosamine > Pierre Millard - Role of the Csr system in carbon nutrition and in the control of central metabolism in Escherichia coli K-12 MG1655 and Nissle 1917
# (Dextrose), Rhamnose, Sorbitol > https://www.tandfonline.com/doi/full/10.3109/08910600903444267
# EcN is a typical E. coli strain, with the exception that it is capable of metabolizing arginine. This property is seen in just about 7% of all E. coli isolates. > https://www.tandfonline.com/doi/full/10.3109/08910600903444267

carbon_sources = ['EX_arab__L_e',
                 'EX_fuc__L_e',
                 'EX_gal_e',
                 'EX_glcn_e',
                 'EX_man_e',
                 'EX_acgal_e',
                 'EX_acnam_e',
                 'EX_galur_e',
                 'EX_xyl__D_e',
                 'EX_ac_e',
                 'EX_fru_e',
                 'EX_mal__D_e',
                 'EX_mal__L_e',
                 'EX_malt_e',
                 'EX_rib__D_e',
                 'EX_glcur_e',
                 'EX_lcts_e',
                 'EX_acgam_e',
                 'EX_rmn_e',
                 'EX_sbt__D_e',
                 'EX_arg__L_e']

print('Number of carbon sources:', len(carbon_sources))

c_source_no_growth = []

for c_source in carbon_sources:
    with model:
        # set glucose to zero, to ensure you only have one carbon source
        model.reactions.EX_glc__D_e.lower_bound=0
        
        # change to other carbon source
        model.reactions.get_by_id(c_source).lower_bound=-20
        
        #determine growth on carbon source
        solution = model.slim_optimize()
        if solution > 0:
            pass
        else:
            c_source_no_growth.append(c_source)
        print(c_source, round(solution, 3))

Number of carbon sources: 21
EX_arab__L_e 0.995
EX_fuc__L_e 0.798
EX_gal_e 1.079
EX_glcn_e 1.049
EX_man_e 1.121
EX_acgal_e 1.155
EX_acnam_e 1.291
EX_galur_e 0.966
EX_xyl__D_e 0.869
EX_ac_e 0.429
EX_fru_e 1.121
EX_mal__D_e nan
EX_mal__L_e 0.764
EX_malt_e 1.594
EX_rib__D_e 0.869
EX_glcur_e 0.966
EX_lcts_e 1.55
EX_acgam_e 1.164
EX_rmn_e 0.798
EX_sbt__D_e 1.071
EX_arg__L_e 0.836


In [8]:
# load EcN & MG1655 model and create a copy
modelCopy = model.copy()
baseCopy = base.copy()
m9(modelCopy)
m9(baseCopy)
modelCopy.reactions.EX_glc__D_e.lower_bound=0 #set glucose to zero, to ensure you only have one carbon source
baseCopy.reactions.EX_glc__D_e.lower_bound=0

c_dict = {}

# adjust the carbon source
for c_source in c_source_no_growth:
    with modelCopy:
        with baseCopy:

            # Change the carbon source
            modelCopy.reactions.get_by_id(c_source).lower_bound=-20
            base.reactions.get_by_id(c_source).lower_bound=-20

            # Define the universal model
            universal = cobra.Model("universal_reactions")
            for rxn_obj in baseCopy.reactions:
                if rxn_obj.id not in modelCopy.reactions:
                    universal.add_reaction(rxn_obj.copy())

            # Run gafiller
            gapfiller = cobra.flux_analysis.gapfilling.GapFiller(modelCopy, universal=universal, demand_reactions=False, integer_threshold=1e-12)
            gapfiller.model.solver.configuration.tolerances.feasibility = 1e-16
            gapfiller.model.solver.configuration.tolerances.integrality = 1e-16
            gapfiller.model.solver.configuration.tolerances.optimality = 1e-16
            result = gapfiller.fill()

            # save the result in a dictionary
            genes = []
            for result_1 in result:
                for gene in result_1:
                    genes.append(gene.id)
#             print(c_source, modelCopy.reactions.get_by_id(c_source).lower_bound, modelCopy.reactions.get_by_id('EX_glc__D_e').lower_bound, modelCopy.reactions.get_by_id('EX_fru_e').lower_bound)
            c_dict[c_source] = genes
    
c_dict

{'EX_mal__D_e': ['MALDDH']}

### Reaction addition
According to literature, EcN can grow on malate. This was also verified by the biolog phenotype experiment. Add reaction "MALDDH" to enable this capacity.

In [9]:
base.reactions.MALDDH

Reaction identifier,MALDDH
Name,Malate decarboxylating oxidoreductase (decarboxylating)
Memory address,0x01dae1f69e08
Stoichiometry,mal__D_c + nad_c --> co2_c + nadh_c + pyr_c D-Malate + Nicotinamide adenine dinucleotide --> CO2 CO2 + Nicotinamide adenine dinucleotide - reduced + Pyruvate
GPR,b1800
Lower bound,0.0
Upper bound,1000.0


In [10]:
base.genes.b1800

Gene identifier,b1800
Name,dmlA
Memory address,0x01dae1f24888
Functional,True
In 2 reaction(s),"IPMD, MALDDH"


_EcoCyc: "Chromosomal expression of dmlA is able to complement cells lacking leuB-encoded 3-isopropylmalate dehydrogenase (IPMDH) activity. Purified DmlA is active on D-malate, 3-isopropylmalate and L(+)-tartrate; D-lactate, isocitrate and D-tartrate are poor substrates. DmlA has a weaker affinity for L(+)-tartrate than for D-malate or 3-isopropylmalate [Vorobieva14]"_

However, there is no gene in EcN with homology to the dmlA gene of MG1655. The reaction will be added, but the gene will not. As a result, the ability of _dmlA_ to complement cells lacking _leuB_ will not be described by this model.

In [11]:
# Ensure that the model version is the latests and unmodified
EcN_model = cobra.io.load_json_model('../data/models/CP022686.1_cur_4.2.json')

# Load the table with information on origin of reactions
rxn_origin_df = pd.read_csv('../tables/rxn_origin.csv')
rxn_origin_df.set_index('reaction', inplace=True)
rxn_origin_df.head()

# Add reaction to model
base.reactions.MALDDH.gene_reaction_rule = ''
EcN_model.add_reactions([base.reactions.MALDDH])

# Update reaction overview dataframe
rxn_origin_df.loc['MALDDH', 'origin'] = 'iML1515'
rxn_origin_df.loc['MALDDH', 'added'] = 'manual'
rxn_origin_df.loc['MALDDH', 'notebook'] = '4.3'
rxn_origin_df.loc['MALDDH', 'notes'] = 'Added in 4.3. to complete a blocked pathway. Due to low homology, no gene was added'

In [12]:
with EcN_model:
    # set glucose to zero, and change to D-malate
    EcN_model.reactions.EX_glc__D_e.lower_bound=0
    EcN_model.reactions.get_by_id('EX_mal__D_e').lower_bound=-20
        
    #determine growth on carbon source
    solution = EcN_model.slim_optimize()

solution

0.6196792055045546

# 3. Save updated model

In [13]:
# Save the model
cobra.io.json.save_json_model(EcN_model, str('../data/models/%s_cur_4.3.json'%EcN_ID), pretty=False)

In [14]:
# Save as a table
rxn_origin_df.to_csv('../tables/rxn_origin.csv')